In [12]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy import stats

stocks = ["AVGO", "COST", "FSLR", "UHS"]
weights = np.array([0.25, 0.25, 0.25, 0.25])
confidence = 0.95
horizon = 15
alpha = 1 - confidence

raw = yf.download(stocks, period="10y", auto_adjust=True)["Close"]
raw.dropna(inplace=True)

returns = raw.pct_change().dropna()
portfolio_returns = returns.dot(weights)

def get_window(returns, years):
    n = int(years * 252)
    return returns.iloc[-n:]

def var_mgb(returns):
    mu = returns.mean()
    sigma = returns.std()
    mu_h = mu * horizon
    sigma_h = sigma * np.sqrt(horizon)
    z = stats.norm.ppf(alpha)
    return -(mu_h + z * sigma_h) * 100

def var_ewma(returns, lam):
    r = returns.values
    var_t = np.var(r)
    for ri in r:
        var_t = lam * var_t + (1 - lam) * ri**2
    mu = returns.mean()
    sigma_h = np.sqrt(var_t) * np.sqrt(horizon)
    mu_h = mu * horizon
    z = stats.norm.ppf(alpha)
    return -(mu_h + z * sigma_h) * 100

years_list = [1, 3, 5, 10]
results = []

for y in years_list:
    w = get_window(portfolio_returns, y)
    results.append({"Método": "MGB", "Lambda": "-", "Ventana": f"{y}Y", "VaR (%)": round(var_mgb(w), 4)})

for lam in [0.94, 0.80]:
    for y in years_list:
        w = get_window(portfolio_returns, y)
        results.append({"Método": "EWMA", "Lambda": lam, "Ventana": f"{y}Y", "VaR (%)": round(var_ewma(w, lam), 4)})

# ─────────────────────────────────────────
# TABLA COMPARATIVA
# ─────────────────────────────────────────
df = pd.DataFrame(results)
pivot = df.pivot_table(index=["Método", "Lambda"], columns="Ventana", values="VaR (%)", aggfunc="first")
pivot = pivot[[f"{y}Y" for y in years_list]]

print("\nVaR (%) — 15 días — 95% confianza")
print("Portafolio: AVGO | COST | FSLR | UHS (25% c/u)\n")
print(pivot.to_string())

[*********************100%***********************]  4 of 4 completed


VaR (%) — 15 días — 95% confianza
Portafolio: AVGO | COST | FSLR | UHS (25% c/u)

Ventana            1Y      3Y      5Y     10Y
Método Lambda                                
EWMA   0.8     5.5984  5.4693  5.7610  6.0180
       0.94    6.3900  6.2609  6.5525  6.8095
MGB    -       9.0939  7.7637  8.1971  8.4565
